Oblv Client is from our PyOblv package.
ds_abc is the PySyft Client for the Data Scientist on domain abc
ds_xyz is the PySyft Client for the Data Scientist on domain xyz

In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import oblv
canada = sy.login(url="localhost",port=8081,email="sheldon@caltech.edu",password="changethis")
italy = sy.login(url="localhost",port=8082,email="sheldon@caltech.edu",password="changethis")


Anyone can login into your account right now because your password is the default PySyft password!!!

Connecting to localhost... done! 	 Logging into canada... done!

Anyone can login into your account right now because your password is the default PySyft password!!!

Connecting to localhost... done! 	 Logging into italy... done!


In [2]:
#Oblivious client facilitates Secure Enclave Deployment
oblv_client = oblv.authenticate("your_api_key_here")

In [3]:
depl = sy.create_deployment(oblv_client,[canada,italy],"demo_openmined_1","first",infra="r5.xlarge")
depl.deployment_id

passed  MIIBCgKCAQEAlW4HN7SA5tfPIidkp/BRgZwOd/ZnjmkyjxGTzee6S3MpVDquiTwY1m/Oa4tU19HDmEeZboMAgG/XSDoArIvvLmP7vtTDaemaLGUCAiLxMeCBdB0qlTVoaqT03M2JuS3LXwdjGPAzoOz+7x0fmM02PrzdNNN2iPhe73S6g7ZWDyGetAV32To15Ht/DvKIZoHguZAf1aBM5LY/YMC6ufODNX04NxGvnv56IF2YAKfkg3slzYwdrxBBdZQiTNgjmcOrVYqYToVk34VFfzc6n6Ms3YmjmK7e1kTvAyPko1PSaDBOXNWnNYeicyQGKkdbuoyVzbF/l9X+6+DZiQXkg+YHLQIDAQAB


'd-5c6cc2c6-8d2e-45d7-aeb8-09baffa489c3'

In [ ]:
# To use an already existing enclave
# depl = sy.oblv.model.DeploymentClient(deployment_id="d-2dfedbb1-7904-493b-8793-1a9554badae7",oblv_client=oblv_client,domain_clients=[canada,italy],user_key_name="first") #connection_port key can be added to set the port on which oblv_proxy will run
# depl.deployment_id

In [5]:
depl_info = oblv_client.deployment_info(depl.deployment_id)
print(depl_info.current_state)

Running


In [6]:
canada_ptr = canada.store[-1]
italy_ptr = italy.store[-1]

In [7]:
depl.__dict__

{'deployment_id': 'd-5c6cc2c6-8d2e-45d7-aeb8-09baffa489c3',
 'user_key_name': 'first',
 'oblv_client': <oblv.oblv_client.OblvClient at 0x107891af0>,
 'client': [<DomainClient - canada: <UID: 78242698ef914e03af8efea4438fbb71>>,
  <DomainClient - italy: <UID: a0674e22a28f43c8a1b6f031d571e5ef>>],
 '_DeploymentClient__conn_string': '',
 '_DeploymentClient__process': None,
 '_DeploymentClient__logs': None}

tensor_abc and tensor_xyz are the two tensors selected from two different domains.

Creating a deploynent now, using both the domains (abc and xyz).

The public-private key pair was already generated for both these domains, but if it is not, the admins of the domain can use the method **create_key** in oblv api to create one.

Deployment with shown id is created. The status of the deployment can be checked with the next block. Once the status is **Running**, the further steps can be taken after an initialization of the connection. 

The port on which the connection is to be made can be specified either during the time of creation of deployment or can be later updated as shown below.

*Do not change the connection port after initializing the connection*

In [8]:
canada.oblv.check_connection(depl)

'Successfully connected to the enclave'

In [9]:
depl.check_proxy_running()

False

In [10]:
depl.initiate_connection(3032)

Successfully connected to proxy on port 3032. The logs can be found at /Users/rasswanth/.oblv_syft_logs/proxy_logs_16_12_2022_15_54_46.log


In [11]:
canada_dataset = canada.oblv.send_dataset(deployment=depl,dataset=canada_ptr)
italy_dataset = italy.oblv.send_dataset(deployment=depl,dataset=italy_ptr)

In [12]:
depl.get_uploaded_datasets()

[{'id': '5dd4043cf7be46a2a3da08439d48e702', 'owner': 'canada'},
 {'id': 'f40063c692a447f7bc9b00255ef1d98e', 'owner': 'italy'}]

In [13]:
type(canada_dataset)

syft.oblv.oblv_tensor_wrapper.OblvTensorWrapper

In [14]:
dataset_result = canada_dataset + italy_dataset
# dataset_result = sy.OblvTensorWrapper(id="r-3533056d-e408-40bc-8550-d6d98d7cfca3",deployment_client=depl)

In [ ]:
# import numpy as np
# arr = np.array([1,2,3,4,5])
# dataset_result = dataset_abc + arr

In [15]:
dataset_sum_result = dataset_result.sum(keepdims=True)
# dataset_result = sy.OblvTensorWrapper(id="r-3533056d-e408-40bc-8550-d6d98d7cfca3",deployment_client=depl)

In [16]:
dataset_sum_result.id

'r-a4f0c414-cbe1-48b0-b6c7-97981c5ad8b4'

In [17]:
# p_id = depl.request_publish(dataset_sum_result.id, sigma=0.5)
p_id=dataset_sum_result.request_publish(sigma=1000)

print(p_id)

p-aef8c3a5-9f5a-4031-a526-9931b584967e


In [18]:
print(canada.privacy_budget)
print(italy.privacy_budget)

9999998.94627288
9999998.94627288


In [19]:
depl.check_publish_request_status(p_id)

Result is ready


In [20]:
print(canada.privacy_budget)
print(italy.privacy_budget)

9999998.89254576
9999998.89254576


In [21]:
result = depl.fetch_result(p_id)
print(result)

[-125.08190333363657]


In [22]:
depl.close_connection()
oblv_client.remove_deployment(depl.deployment_id)

{
  "message": "Delete Deployment Initiated"
}

In [23]:
italy.oblv.get_key()

'MIIBCgKCAQEA07ia/xeYHaiWCNH+P5pqugUoC3cliY1jbpR2QjN0Saxgc/tbTVDSfW2ac08zynyCYB2OzfYgStG+VSol+eNRjcmc/StCeMZlxjOL0c7KVejDI/p1SfCgyqHYugd6ajeXIcXStU68mxsI1NsivCXW9UCT1WHAXY1sq7R5bg2WazmcGDSFyXceA0kX2AQ045Pc3q5uozwpRPk09sIUIPJZEmEl/KM0nlRCpuvT3OQB3jsaNQJRV0WpB5wI9oS3MT9tMUuVBLIJJ9ni93T+6ViHSrA0+b/3cCb999SUI6Hb2/t9L3y3GDBbJ4LqVVsbndo+RTk0QYcdDbw746Fic6L0oQIDAQAB'

In [24]:
depl.fetch_current_proxy_logs()

Configuration file stored at: "/Users/rasswanth/Library/Application Support/oblv/oblv_config.yaml"

Saved connect command. Use reconnect for reusing cached command.
[2022-12-16T10:24:48Z INFO  lib_ccli::signing] Creating client hello message for handshake with enclave...
[2022-12-16T10:24:48Z INFO  lib_ccli] Establishing connection to enclave https://conso-appli-cakogfo83hif-1195392546.enclave.oblivious.ai:443
[2022-12-16T10:24:48Z INFO  lib_ccli::communication] Sending client hello message to enclave...
[2022-12-16T10:24:50Z INFO  lib_ccli] Received attestation document from enclave.
[2022-12-16T10:24:50Z INFO  lib_ccli] Verifying the received attestation document...
[2022-12-16T10:24:50Z INFO  lib_ccli::attestation_doc] ===> Validate if module id is empty.
[2022-12-16T10:24:50Z INFO  lib_ccli::attestation_doc] ===> Validate digest signature.
[2022-12-16T10:24:50Z INFO  lib_ccli::attestation_doc] ===> Validate PCR map length.
[2022-12-16T10:24:50Z INFO  lib_ccli::attestation_doc] ===>

Known Issues - 

Large dataset cannot be used as of now, cause they go in long re-runs for cache calculation.

Proxy sends 200 status code while the enclave returns 400.

Mean method when performed on tensor returns a tensor with data_subjects not of type of array and it fails when we perform publish on it.
